In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
78863,Эсватини,2021-01-26,14830,9542,517,208,148,24
78864,Папский Престол,2020-04-05,7,0,0,0,0,0
78865,Исландия,2020-05-28,1805,1792,10,0,0,0
78866,Гвинея,2020-03-30,22,0,0,6,0,0
78867,Алжир,2020-10-01,51690,36282,1741,160,108,5
78868,Уругвай,2020-03-31,338,41,1,18,41,0
78869,Камбоджа,2020-08-01,240,196,0,1,32,0
78870,США,2020-05-04,1191677,187180,72432,24085,7028,1378
78871,Норвегия,2020-03-30,4445,12,32,161,5,7
78872,Гвинея-Бисау,2020-09-06,2245,1127,38,0,0,0


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
403,Весь мир,2021-02-28,114109186,64418459,2531755,303177,175176,5379
404,Весь мир,2021-03-01,114415400,64633384,2538961,306214,214925,7206
405,Весь мир,2021-03-02,114725863,64903489,2549372,310463,270105,10411
406,Весь мир,2021-03-03,115166644,65132897,2560422,440781,229408,11050
407,Весь мир,2021-03-04,115619554,65383177,2570176,452910,250280,9754
408,Весь мир,2021-03-05,116066168,65609537,2580166,446614,226360,9990
409,Весь мир,2021-03-06,116476413,65846038,2587977,410245,236501,7811
410,Весь мир,2021-03-07,116844471,66084360,2593224,368058,238322,5247
411,Весь мир,2021-03-08,117142769,66358917,2600063,298298,274557,6839
412,Весь мир,2021-03-09,117557807,66621151,2610184,415038,262234,10121


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2021-03-09


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Marshall Islands'), (1, 'Micronesia'), (2, 'Австралия'), (3, 'Австрия'), (4, 'Азербайджан'), (5, 'Албания'), (6, 'Алжир'), (7, 'Ангола'), (8, 'Андорра'), (9, 'Антигуа и Барбуда'), (10, 'Аргентина'), (11, 'Армения'), (12, 'Афганистан'), (13, 'Багамские острова'), (14, 'Бангладеш'), (15, 'Барбадос'), (16, 'Бахрейн'), (17, 'Беларусь'), (18, 'Белиз'), (19, 'Бельгия'), (20, 'Бенин'), (21, 'Болгария'), (22, 'Боливия'), (23, 'Босния и Герцеговина'), (24, 'Ботсвана'), (25, 'Бразилия'), (26, 'Бруней'), (27, 'Буркина-Фасо'), (28, 'Бурунди'), (29, 'Бутан'), (30, 'Вануату'), (31, 'Великобритания'), (32, 'Венгрия'), (33, 'Венесуэла'), (34, 'Весь мир'), (35, 'Восточный Тимор'), (36, 'Вьетнам'), (37, 'Габон'), (38, 'Гаити'), (39, 'Гайана'), (40, 'Гамбия'), (41, 'Гана'), (42, 'Гватемала'), (43, 'Гвинея'), (44, 'Гвинея-Бисау'), (45, 'Германия'), (46, 'Гондурас'), (47, 'Гренада'), (48, 'Греция'), (49, 'Грузия'), (50, 'ДР Конго'), (51, 'Дания'), (52, 'Джибути'), (53, 'Доминика'), (54, 'Д

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
79291,2021-03-05,437222,415868,8190,1129,2690,55,1.122,1.009,0,0,Япония,13164,1049.857143,1036.000000,14310.857143,15886.285714,-20,-1616
79292,2021-03-06,437634,417136,8196,412,1268,6,1.028,1.009,0,0,Япония,12302,934.428571,977.285714,13872.571429,15380.928571,-717,-862
79293,2021-03-07,439356,418105,8255,1722,969,59,1.146,1.010,0,0,Япония,12996,1038.000000,1026.285714,13580.857143,14970.785714,1310,694
79294,2021-03-08,439977,418105,8301,621,0,46,0.970,1.010,0,0,Япония,13571,1028.428571,1018.000000,13463.857143,14648.000000,-1101,575
79295,2021-03-09,441152,420104,8359,1175,1999,58,0.884,1.009,0,0,Япония,12689,1064.571429,1023.142857,13314.285714,14298.642857,554,-882


In [6]:
# Добавляем колонки расчитанные ,доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2021-03-08  117142769       48183789
2021-03-09  117557807       48326472
79296
(79296, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
54083,2021-02-18,4079407,Россия,3.698138,0.826738
54084,2021-02-19,4092649,Россия,3.696326,0.811371
54085,2021-02-20,4105424,Россия,3.695509,0.798881
54086,2021-02-21,4117992,Россия,3.696371,0.789472
54087,2021-02-22,4130447,Россия,3.697977,0.786168
54088,2021-02-23,4142126,Россия,3.695594,0.781122
54089,2021-02-24,4153735,Россия,3.691313,0.775400
54090,2021-02-25,4164802,Россия,3.686502,0.761151
54091,2021-02-26,4175757,Россия,3.681764,0.747869
54092,2021-02-27,4187166,Россия,3.679213,0.735661


In [8]:
X.to_csv('W.csv',index=False)